In [1]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

import numpy as np
import matplotlib.pyplot as plt
from dataclasses import dataclass

import json
import sys
import numpy.random as random
sys.path.append("../../")

import time 

from src.online_prediction.OnlineDataProcessor.dataProcessor import DataProcessor

In [ ]:
@dataclass
class config:
    window_length: int = 20
    smooth_fc: float = 1.5
    smooth_order: int = 3
    Ts : float = 0.01
    min_vals: float = -2.0
    max_vals: float = 3.0


dataProcesoor = DataProcessor(config)


sine_data = np.sin(np.arange(0, 200, 0.1))

last_time = time.time()

data_received = []
traffic_list = []
count = 1
while True:
    if time.time() - last_time > 0.01:
        data_point = sine_data[count]
        if random.rand() > 0.5:
            processed_data = dataProcesoor.add_data_point(data_point)
        last_time = time.time()

        if count % config.window_length == 0:
            window_data, packet_count = dataProcesoor.get_window_data()
            data_received.append(window_data)
            traffic_list.append(packet_count)
        count += 1
    
    if count >= len(sine_data):
        break

window_data = np.array(data_received).reshape(-1)
traffic_list = np.array(traffic_list).reshape(-1)
plt.plot(window_data)
plt.plot(sine_data[:len(window_data)])
plt.show()
plt.plot(traffic_list)
plt.show()